In [1]:
import os
import argparse
import math
from decimal import Decimal
from os.path import join

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats
import pandas as pd
from tqdm import tqdm

from tractseg.data import dataset_specific_utils
from tractseg.libs.AFQ_MultiCompCorrection import AFQ_MultiCompCorrection
from tractseg.libs.AFQ_MultiCompCorrection import get_significant_areas
from tractseg.libs import metric_utils
from tractseg.libs import plot_utils
from tractseg.libs import tracking

ImportError: dlopen: cannot load any more object with static TLS

In [3]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
from os.path import join
import argparse

import nibabel as nib
import numpy as np
from nibabel import trackvis
from tqdm import tqdm

from tractseg.libs import tractometry
from tractseg.data import dataset_specific_utils


'/NAS/dumbo/protocoles/CogPhenoPark/TractSeg/nodif_brain_mask.nii.gz/MD_tractometry_72.csv'

In [26]:
import glob
STUDY_PATH='/NAS/dumbo/protocoles/CogPhenoPark/TractSeg/*'
dir_names=glob.glob(STUDY_PATH)
for SUBJECT in dir_names :
    for para in ['FA','density','MD','L1','L2','L3'] : 
        print(SUBJECT)
        print(para)
        results = []
        tracking_dir=os.path.join(SUBJECT,'FOD_iFOD2_trackings/')
        endings_dir=os.path.join(SUBJECT,'endings_segmentations/')
        tracking_format='tck'
        tmp_str=para+'_tractometry_72.csv'                                        
        csv_file_out=os.path.join(SUBJECT,tmp_str)           
        tmp_str='fit_'+para+'.nii.gz'                                                                         
        scalar_image = nib.load(os.path.join(SUBJECT,tmp_str))
        bundles = dataset_specific_utils.get_bundle_names("All")[1:]    
        NR_POINTS=102
        DILATION = 2
        for bundle in tqdm(bundles):
            if False : #args.peak_length:
                predicted_peaks = nib.load(join(TOM_dir, bundle + ".nii.gz")).get_data()
            else:
                predicted_peaks = None
            beginnings = nib.load(join(endings_dir, bundle + "_b.nii.gz"))

            file_ending = "trk" if tracking_format == "trk_legacy" else tracking_format
            trk_path = join(tracking_dir, bundle + "." + file_ending)

            if not os.path.exists(trk_path):
                print("WARNING: No tracking found for bundle {}. Returning zeros.".format(bundle))
                mean = np.zeros(NR_POINTS)
                std = np.zeros(NR_POINTS)
            else:
                if tracking_format == "trk_legacy":
                    streams, hdr = trackvis.read(trk_path)
                    streamlines = [s[0] for s in streams]
                else:
                    sl_file = nib.streamlines.load(trk_path)
                    streamlines = sl_file.streamlines

                if len(streamlines) >= 5 :
                    mean, std = tractometry.evaluate_along_streamlines(np.nan_to_num(scalar_image.get_data()), streamlines,
                                                                   beginnings.get_data(), NR_POINTS, dilate=DILATION,
                                                                   predicted_peaks=predicted_peaks, affine=scalar_image.affine)
                else:
                    print("WARNING: bundle {} contains less than 5 streamlines. Saving value 0 for this bundle.".
                          format(bundle))
                    mean = np.zeros(NR_POINTS)
                    std = np.zeros(NR_POINTS)

            # Remove first and last segment as those tend to be more noisy
            mean = mean[1:-1]
            std = std[1:-1]

            results.append(mean)

        bundle_string = ""
        for bundle in bundles:
            bundle_string += bundle + ";"
        bundle_string = bundle_string[:-1]

        np.savetxt(csv_file_out, np.array(results).transpose(), delimiter=";", header=bundle_string, comments="")

/NAS/dumbo/protocoles/CogPhenoPark/TractSeg/430702JG100214
FA





  0%|          | 0/72 [00:00<?, ?it/s]/home/global/anaconda37/lib/python3.7/site-packages/ipykernel_launcher.py:42: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
/home/global/anaconda37/lib/python3.7/site-packages/ipykernel_launcher.py:43: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0



  1%|▏         | 1/72 [00:08<10:22,  8.77s/it]


  3%|▎         | 2/72 [00:12<08:22,  7.17s/it]


  4%|▍         | 3/72 [00:23<09:40,  8.41s/it]


  6%|▌         | 4/72 [00:32<09:35,

/NAS/dumbo/protocoles/CogPhenoPark/TractSeg/430702JG100214
density





  1%|▏         | 1/72 [00:04<05:48,  4.90s/it]


  3%|▎         | 2/72 [00:08<05:09,  4.43s/it]


  4%|▍         | 3/72 [00:17<06:51,  5.96s/it]


  6%|▌         | 4/72 [00:25<07:15,  6.41s/it]


  8%|▊         | 6/72 [00:32<06:03,  5.51s/it]


 10%|▉         | 7/72 [01:01<13:48, 12.75s/it]


 11%|█         | 8/72 [01:03<10:08,  9.50s/it]


 12%|█▎        | 9/72 [01:06<07:54,  7.53s/it]


 14%|█▍        | 10/72 [02:00<22:05, 21.38s/it]


 15%|█▌        | 11/72 [03:35<44:17, 43.56s/it]


 17%|█▋        | 12/72 [03:43<32:52, 32.88s/it]


 18%|█▊        | 13/72 [03:59<27:28, 27.94s/it]


 19%|█▉        | 14/72 [04:15<23:20, 24.14s/it]


 21%|██        | 15/72 [04:16<16:32, 17.41s/it]


 22%|██▏       | 16/72 [04:17<11:30, 12.32s/it]


 24%|██▎       | 17/72 [06:19<41:26, 45.21s/it]


 25%|██▌       | 18/72 [08:22<1:01:48, 68.67s/it]


 26%|██▋       | 19/72 [08:24<42:52, 48.54s/it]  


 28%|██▊       | 20/72 [08:25<29:40, 34.24s/it]


 29%|██▉       | 21/72 [08:26<20:36, 24.24s/it]


 

/NAS/dumbo/protocoles/CogPhenoPark/TractSeg/430702JG100214
MD


...




 28%|██▊       | 20/72 [02:47<09:58, 11.51s/it]




 32%|███▏      | 23/72 [02:49<06:45,  8.27s/it]


 33%|███▎      | 24/72 [02:51<04:59,  6.24s/it]


 35%|███▍      | 25/72 [02:51<03:28,  4.44s/it]


 36%|███▌      | 26/72 [02:51<02:28,  3.22s/it]


 38%|███▊      | 27/72 [02:55<02:27,  3.27s/it]


 39%|███▉      | 28/72 [02:56<01:57,  2.68s/it]


 40%|████      | 29/72 [03:09<04:09,  5.81s/it]


 42%|████▏     | 30/72 [03:10<03:06,  4.44s/it]


 43%|████▎     | 31/72 [03:11<02:15,  3.30s/it]


 44%|████▍     | 32/72 [03:13<01:53,  2.83s/it]


 46%|████▌     | 33/72 [03:15<01:46,  2.74s/it]


 47%|████▋     | 34/72 [03:20<02:09,  3.42s/it]


 49%|████▊     | 35/72 [03:23<02:00,  3.26s/it]


 50%|█████     | 36/72 [03:27<02:07,  3.53s/it]


 51%|█████▏    | 37/72 [03:29<01:46,  3.05s/it]


 53%|█████▎    | 38/72 [03:30<01:16,  2.26s/it]


 54%|█████▍    | 39/72 [03:30<01:00,  1.84s/it]


 56%|█████▌    | 40/72 [03:34<01:15,  2.36s/it]


 57%|█████▋    | 41/72 [03:38<01:24,  2.73s/it]


 58%|█████▊    | 42/72 [03:38<01:03,  2.11s/it]

/NAS/dumbo/protocoles/CogPhenoPark/TractSeg/340910RL270314
density





  1%|▏         | 1/72 [00:05<06:20,  5.36s/it]


  3%|▎         | 2/72 [00:06<04:39,  3.99s/it]


  4%|▍         | 3/72 [00:10<04:35,  4.00s/it]

KeyboardInterrupt: 



  0%|          | 0/72 [00:00<?, ?it/s]/home/global/anaconda37/lib/python3.7/site-packages/ipykernel_launcher.py:33: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
/home/global/anaconda37/lib/python3.7/site-packages/ipykernel_launcher.py:34: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0


  1%|▏         | 1/72 [00:26<30:49, 26.04s/it]

  3%|▎         | 2/72 [00:43<27:19, 23.42s/it]

  4%|▍         | 3/72 [00:55<23:06, 20.10s/it]

  6%|▌         | 4/72 [01:14<22:18, 19.6

KeyboardInterrupt: 

In [31]:
def parse_subjects_file(file_path):
    with open(file_path) as f:
        l = f.readline().strip()
        if l.startswith("# tractometry_path="):
            base_path = l.split("=")[1]
        else:
            raise ValueError("Invalid first line in subjects file. Must start with '# tractometry_path='")

        bundles = None
        plot_3D_path = None

        # parse bundle names
        for i in range(2):
            l = f.readline().strip()
            if l.startswith("# bundles="):
                bundles_string = l.split("=")[1]
                bundles = bundles_string.split(" ")

                valid_bundles = dataset_specific_utils.get_bundle_names("All_tractometry")[1:]
                for bundle in bundles:
                    if bundle not in valid_bundles:
                        raise ValueError("Invalid bundle name: {}".format(bundle))

                print("Using {} manually specified bundles.".format(len(bundles)))
            elif l.startswith("# plot_3D="):
                plot_3D_path = l.split("=")[1]

        if bundles is None:
            bundles = dataset_specific_utils.get_bundle_names("All_tractometry")[1:]

    df = pd.read_csv(file_path, sep=" ", comment="#")
    df["subject_id"] = df["subject_id"].astype(str)

    # Check that each column (except for first one) is correctly parsed as a number
    for col in df.columns[1:]:
        if not np.issubdtype(df[col].dtype, np.number):
            raise IOError("Column {} contains non-numeric values".format(col))

    #if df.columns[1] == "group":
    #    if df["group"].max() > 1:
    #        raise IOError("Column 'group' may only contain 0 and 1.")

    return base_path, df, bundles, plot_3D_path

In [3]:
def correct_for_confounds(values, meta_data, bundles, selected_bun_indices, NR_POINTS, analysis_type, confound_names):
    values_cor = np.zeros([len(bundles), NR_POINTS, len(meta_data)])
    for b_idx in selected_bun_indices:
        for jdx in range(NR_POINTS):
            target = np.array([values[s][b_idx][jdx] for s in meta_data["subject_id"]])
            if analysis_type == "group":
                target_cor = metric_utils.unconfound(target, meta_data[["group"] + confound_names].values,
                                                     group_data=True)
            else:
                target_cor = metric_utils.unconfound(target, meta_data[confound_names].values,
                                                     group_data=False)
                meta_data["target"] = metric_utils.unconfound(meta_data["target"].values,
                                                              meta_data[confound_names].values,
                                                              group_data=False)
            values_cor[b_idx, jdx, :] = target_cor

    # Restore original data structure
    values_cor = values_cor.transpose(2, 0, 1)
    # todo: nicer way: use numpy array right from beginning instead of dict
    values_cor_dict = {}
    for idx, subject in enumerate(list(meta_data["subject_id"])):
        values_cor_dict[subject] = values_cor[idx]
    return values_cor_dict

In [4]:
def get_corrected_alpha(values_allp, meta_data, analysis_type, subjects_A, subjects_B, alpha, bundles, nperm, b_idx):
    if analysis_type == "group":
        y = np.array((0,) * len(subjects_A) + (1,) * len(subjects_B))
    else:
        y = meta_data["target"].values
    alphaFWE, statFWE, clusterFWE, stats = AFQ_MultiCompCorrection(np.array(values_allp), y,
                                                                   alpha, nperm=nperm)
    #alphaFWE=0.05                                                               
    print("Processing {}...".format(bundles[b_idx]))
    print("  cluster size: {}".format(clusterFWE))
    print("  alphaFWE: {}".format(format_number(alphaFWE)))
    return alphaFWE, clusterFWE

In [5]:
def format_number(num):
    if abs(num) > 0.00001:
        return round(num, 6)
    else:
        return '%.2e' % Decimal(num)

In [6]:
def plot_tractometry_with_pvalue(values, meta_data, bundles, selected_bundles, output_path, alpha, FWE_method,
                                 analysis_type, correct_mult_tract_comp, show_detailed_p, nperm=1000,
                                 hide_legend=False, plot_3D_path=None, plot_3D_type="none",
                                 tracking_format="trk_legacy", tracking_dir="auto", show_color_bar=True):

    NR_POINTS = values[meta_data["subject_id"][0]].shape[1]
    selected_bun_indices = [bundles.index(b) for b in selected_bundles]

    if analysis_type == "group":
        subjects_A = list(meta_data[meta_data["group"] == 0]["subject_id"])
        subjects_B = list(meta_data[meta_data["group"] == 1]["subject_id"])
    else:
        subjects_A = list(meta_data["subject_id"])
        subjects_B = []

    confound_names = list(meta_data.columns[2:])

    cols = 5
    rows = math.ceil(len(selected_bundles) / cols)

    a4_dims = (cols*3, rows*5)
    f, axes = plt.subplots(rows, cols, figsize=a4_dims)

    axes = axes.flatten()
    sns.set(font_scale=1.2)
    sns.set_style("whitegrid")

    # Correct for confounds
    values = correct_for_confounds(values, meta_data, bundles, selected_bun_indices, NR_POINTS, analysis_type,
                                   confound_names)

    # Significance testing with multiple correction of bundles
    if correct_mult_tract_comp:
        values_allp = []  # [subjects, NR_POINTS * nr_bundles]
        for s in meta_data["subject_id"]:
            values_subject = []
            for i, b_idx in enumerate(selected_bun_indices):
                values_subject += list(values[s][b_idx]) # concatenate all bundles
            values_allp.append(values_subject)
        alphaFWE, clusterFWE = get_corrected_alpha(values_allp, meta_data, analysis_type, subjects_A, subjects_B, alpha,
                                                   bundles, nperm, b_idx)

    for i, b_idx in enumerate(tqdm(selected_bun_indices)):
        # Bring data into right format for seaborn
        data = {"position": [],
                "fa": [],
                "group": [],
                "subject": []}
        for j, subject in enumerate(subjects_A + subjects_B):
            for position in range(NR_POINTS):
                data["position"].append(position)
                data["subject"].append(subject)
                data["fa"].append(values[subject][b_idx][position])
                if subject in subjects_A:
                    data["group"].append("Group 0")
                else:
                    data["group"].append("Group 1")

        # Plot
        ax = sns.lineplot(x="position", y="fa", data=data, ax=axes[i], hue="group")
                          # units="subject", estimator=None, lw=1)  # each subject as single line

        ax.set(xlabel='position along tract', ylabel='metric')
        ax.set_title(bundles[b_idx])
        if analysis_type == "correlation" or hide_legend:
            ax.legend_.remove()
        elif analysis_type == "group" and i > 0:
            ax.legend_.remove()  # only show legend on first subplot


        # Significance testing without multiple correction of bundles
        if not correct_mult_tract_comp:
            values_allp = [values[s][b_idx] for s in subjects_A + subjects_B]  # [subjects, NR_POINTS]
            alphaFWE, clusterFWE = get_corrected_alpha(values_allp, meta_data, analysis_type, subjects_A, subjects_B,
                                                       alpha, bundles, nperm, b_idx)

        # Calc p-values
        pvalues = np.zeros(NR_POINTS)
        stats = np.zeros(NR_POINTS)  # for ttest: t-value, for pearson: correlation
        for jdx in range(NR_POINTS):
            if analysis_type == "group":
                values_controls = [values[s][b_idx][jdx] for s in subjects_A]
                values_patients = [values[s][b_idx][jdx] for s in subjects_B]
                stats[jdx], pvalues[jdx] = scipy.stats.ttest_ind(values_controls, values_patients)
            else:
                values_controls = [values[s][b_idx][jdx] for s in subjects_A]
                stats[jdx], pvalues[jdx] = scipy.stats.pearsonr(values_controls, meta_data["target"].values)


        # Plot significant areas
        if show_detailed_p:
            ax2 = axes[i].twinx()
            ax2.bar(range(len(pvalues)), -np.log10(pvalues), color="gray", edgecolor="none", alpha=0.5)
            ax2.plot([0, NR_POINTS-1], (-np.log10(alphaFWE),)*2, color="red", linestyle=":")
            ax2.set(xlabel='position', ylabel='-log10(p)')
        else:
            if FWE_method == "alphaFWE":
                sig_areas = get_significant_areas(pvalues, 1, alphaFWE)
            else:
                sig_areas = get_significant_areas(pvalues, clusterFWE, alpha)
            sig_areas = sig_areas * np.quantile(np.array(data["fa"]), 0.98)
            sig_areas[sig_areas == 0] = np.quantile(np.array(data["fa"]), 0.02)
            axes[i].plot(range(len(sig_areas)), sig_areas, color="red", linestyle=":")

        # Plot text
        if FWE_method == "alphaFWE":
            axes[i].annotate("alphaFWE:   {}".format(format_number(alphaFWE)),
                             (0, 0), (0, -35), xycoords='axes fraction', textcoords='offset points', va='top',
                             fontsize=10)
            axes[i].annotate("min p-value: {}".format(format_number(pvalues.min())),
                             (0, 0), (0, -45), xycoords='axes fraction', textcoords='offset points', va='top',
                             fontsize=10)
        else:
            axes[i].annotate("clusterFWE:   {}".format(clusterFWE),
                             (0, 0), (0, -35), xycoords='axes fraction', textcoords='offset points', va='top',
                             fontsize=10)

        stats_label = "t-value:      " if analysis_type == "group" else "corr.coeff.: "
        axes[i].annotate(stats_label + "   {}".format(format_number(stats[pvalues.argmin()])),
                         (0, 0), (0, -55), xycoords='axes fraction', textcoords='offset points', va='top',
                         fontsize=10)

        if plot_3D_type != "none":

            if plot_3D_type == "metric":
                metric = np.array([values[s][b_idx] for s in subjects_A + subjects_B]).mean(axis=0)
            else:
                # metric = pvalues  # use this code if you want to plot the pvalues instead of the FA
                metric = sig_areas

            bundle = bundles[b_idx]
            output_path_3D = output_path.split(".")[0] + "_" + bundle + "_3D.png"

            if tracking_dir == "auto":
                tracking_dir = tracking.get_tracking_folder_name("fixed_prob", False)

            if tracking_format == "tck":
                tracking_path = join(plot_3D_path, tracking_dir, bundle + ".tck")
            else:
                tracking_path = join(plot_3D_path, tracking_dir, bundle + ".trk")
            ending_path = join(plot_3D_path, "endings_segmentations", bundle + "_b.nii.gz")
            mask_path = join(plot_3D_path, "..", "nodif_brain_mask.nii.gz")

            if not os.path.isfile(tracking_path):
                raise ValueError("Could not find: " + tracking_path)
            if not os.path.isfile(ending_path):
                raise ValueError("Could not find: " + ending_path)
            if not os.path.isfile(mask_path):
                raise ValueError("Could not find: " + mask_path)

            plot_utils.plot_bundles_with_metric(tracking_path, ending_path, mask_path, bundle, metric,
                                                output_path_3D, tracking_format, show_color_bar)


    plt.tight_layout()
    plt.savefig(output_path, dpi=200)

In [1]:
FWE_method = "alphaFWE"
show_detailed_p = False
hide_legend = False
show_color_bar = True  # colorbar on 3D plot
nperm = 5000
nperm = int(nperm / 5)
correct_mult_tract_comp = False
base_path, meta_data, selected_bundles, plot_3D_path = parse_subjects_file("/NAS/dumbo/protocoles/CogPhenoPark/Tractometry_MD.txt")
analysis_type = "group"

NameError: name 'parse_subjects_file' is not defined

In [53]:
all_bundles = dataset_specific_utils.get_bundle_names("All_tractometry")[1:]
values = {}
for subject in meta_data["subject_id"]:
    raw = np.loadtxt(base_path.replace("SUBJECT_ID", subject), delimiter=";", skiprows=1).transpose()
    values[subject] = raw

In [54]:
selected_bundles

['AF_left',
 'AF_right',
 'ATR_left',
 'ATR_right',
 'CG_left',
 'CG_right',
 'CC_1',
 'CC_2',
 'CC_3',
 'CC_4',
 'CC_5',
 'CC_6',
 'CC_7',
 'IFO_left',
 'IFO_right',
 'ILF_left',
 'ILF_right',
 'OR_left',
 'OR_right',
 'SLF_I_left',
 'SLF_I_right',
 'SLF_II_left',
 'SLF_II_right',
 'SLF_III_left',
 'SLF_III_right',
 'STR_left',
 'STR_right',
 'UF_left',
 'UF_right',
 'T_PREM_left',
 'T_PREM_right',
 'T_PAR_left',
 'T_PAR_right',
 'T_OCC_left',
 'T_OCC_right',
 'ST_FO_left',
 'ST_FO_right',
 'ST_PREM_left',
 'ST_PREM_right']

In [22]:
plot_tractometry_with_pvalue(values, meta_data, all_bundles,selected_bundles,"/NAS/dumbo/protocoles/CogPhenoPark/",
                             0.05, FWE_method, analysis_type, correct_mult_tract_comp,
                             show_detailed_p, nperm=nperm, hide_legend=hide_legend,
                             plot_3D_path=plot_3D_path, plot_3D_type="pval",
                             tracking_format="tck", tracking_dir="auto",
                             show_color_bar=show_color_bar)

  0%|          | 0/1 [00:00<?, ?it/s]/home/global/anaconda3/lib/python3.5/site-packages/scipy/stats/stats.py:1713: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.add.reduce(sorted[indexer] * weights, axis=axis) / sumval


Processing ST_FO_left...
  cluster size: 27.0
  alphaFWE: 0.05


100%|██████████| 1/1 [00:11<00:00, 11.28s/it]


In [55]:
NR_POINTS = values[meta_data["subject_id"][0]].shape[1]
selected_bun_indices = [all_bundles.index(b) for b in selected_bundles]

if analysis_type == "group":
    subjects_A = list(meta_data[meta_data["group"] == 0]["subject_id"])
    subjects_B = list(meta_data[meta_data["group"] == 1]["subject_id"])
    subjects_C = list(meta_data[meta_data["group"] == 2]["subject_id"])
    subjects_D = list(meta_data[meta_data["group"] == 3]["subject_id"])
else:
    subjects_A = list(meta_data["subject_id"])
    subjects_B = []

confound_names = list(meta_data.columns[2:])

cols = 3
rows = math.ceil(len(selected_bundles) / cols)

a4_dims = (cols*3, rows*5)
f, axes = plt.subplots(rows, cols, figsize=a4_dims)

axes = axes.flatten()
sns.set(font_scale=1.2)
sns.set_style("whitegrid")

# Correct for confounds
values = correct_for_confounds(values, meta_data, all_bundles, selected_bun_indices, NR_POINTS, analysis_type,confound_names)

for i, b_idx in enumerate(tqdm(selected_bun_indices)):
    
    # Bring data into right format for seaborn
    data = {"position": [],
            "fa": [],
            "group": [],
            "subject": []}
    for j, subject in enumerate(subjects_A + subjects_B + subjects_C + subjects_D ):
        for position in range(NR_POINTS):
            data["position"].append(position)
            data["subject"].append(subject)
            data["fa"].append(values[subject][b_idx][position])
            if subject in subjects_A:
                data["group"].append("Group 0")
            elif subject in subjects_B:
                data["group"].append("Group 1")
            elif subject in subjects_C:
                data["group"].append("Group 2")                
            else:
                data["group"].append("Group 3")

    # Plot
    ax = sns.lineplot(x="position", y="fa", data=data, ax=axes[i], hue="group")
                      # units="subject", estimator=None, lw=1)  # each subject as single line
    print(all_bundles[b_idx])
    ax.set(xlabel='position along tract', ylabel='metric')
    ax.set_title(all_bundles[b_idx])
    if analysis_type == "correlation" or hide_legend:
        ax.legend_.remove()
    elif analysis_type == "group" and i > 0:
        ax.legend_.remove()  # only show legend on first subplot

    alpha=0.05
    nperm=1000
    # Significance testing without multiple correction of bundles
    if not correct_mult_tract_comp:
        values_allp = [values[s][b_idx] for s in subjects_A + subjects_B + subjects_C + subjects_D ]  # [subjects, NR_POINTS]
        #alphaFWE, clusterFWE = get_corrected_alpha(values_allp, meta_data, analysis_type, subjects_A, subjects_B,alpha,all_bundles, nperm, b_idx)
        alphaFWE=0.05

    # Calc p-values
    pvalues = np.zeros(NR_POINTS)
    stats = np.zeros(NR_POINTS)  # for ttest: t-value, for pearson: correlation
    for jdx in range(NR_POINTS):
        if analysis_type == "group":
            values_A = [values[s][b_idx][jdx] for s in subjects_A]
            values_B = [values[s][b_idx][jdx] for s in subjects_B]
            values_C = [values[s][b_idx][jdx] for s in subjects_C]
            values_D = [values[s][b_idx][jdx] for s in subjects_D]
            stats[jdx], pvalues[jdx] = scipy.stats.kruskal(values_A, values_B,values_C, values_D)
        else:
            values_controls = [values[s][b_idx][jdx] for s in subjects_A]
            stats[jdx], pvalues[jdx] = scipy.stats.pearsonr(values_controls, meta_data["target"].values)

    # Plot significant areas
    if show_detailed_p:
        ax2 = axes[i].twinx()
        ax2.bar(range(len(pvalues)), -np.log10(pvalues), color="gray", edgecolor="none", alpha=0.5)
        ax2.plot([0, NR_POINTS-1], (-np.log10(alphaFWE),)*2, color="red", linestyle=":")
        ax2.set(xlabel='position', ylabel='-log10(p)')
    else:
        if FWE_method == "alphaFWE":
            sig_areas = get_significant_areas(pvalues, 1, alphaFWE)
        else:
            sig_areas = get_significant_areas(pvalues, clusterFWE, alpha)
        sig_areas = sig_areas * np.quantile(np.array(data["fa"]), 0.98)
        sig_areas[sig_areas == 0] = np.quantile(np.array(data["fa"]), 0.02)
        axes[i].plot(range(len(sig_areas)), sig_areas, color="red", linestyle=":")

    # Plot text
    if FWE_method == "alphaFWE":
        axes[i].annotate("alphaFWE:   {}".format(format_number(alphaFWE)),
                         (0, 0), (0, -35), xycoords='axes fraction', textcoords='offset points', va='top',
                         fontsize=10)
        axes[i].annotate("min p-value: {}".format(format_number(pvalues.min())),
                         (0, 0), (0, -45), xycoords='axes fraction', textcoords='offset points', va='top',
                         fontsize=10)
    else:
        axes[i].annotate("clusterFWE:   {}".format(clusterFWE),
                         (0, 0), (0, -35), xycoords='axes fraction', textcoords='offset points', va='top',
                         fontsize=10)

    stats_label = "t-value:      " if analysis_type == "group" else "corr.coeff.: "
    axes[i].annotate(stats_label + "   {}".format(format_number(stats[pvalues.argmin()])),
                     (0, 0), (0, -55), xycoords='axes fraction', textcoords='offset points', va='top',
                     fontsize=10)
        
plt.tight_layout()
plt.savefig("/NAS/dumbo/protocoles/CogPhenoPark/GroupComparison_AllBundle_MD.png", dpi=200)

/home/global/anaconda3/lib/python3.5/site-packages/matplotlib/pyplot.py:513: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)
  0%|          | 0/39 [00:00<?, ?it/s]/home/global/anaconda3/lib/python3.5/site-packages/scipy/stats/stats.py:1713: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.add.reduce(sorted[indexer] * weights, axis=axis) / sumval
  3%|▎         | 1/39 [00:04<02:44,  4.34s/it]

AF_left


  5%|▌         | 2/39 [00:08<02:40,  4.33s/it]

AF_right


  8%|▊         | 3/39 [00:12<02:35,  4.32s/it]

...

ST_FO_right


 97%|█████████▋| 38/39 [02:43<00:04,  4.31s/it]

ST_PREM_left


100%|██████████| 39/39 [02:48<00:00,  4.31s/it]

ST_PREM_right
